In [131]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

# MNIST 
Creating a trained netword for testing

In [148]:
# MNIST neural net
class MNIST(nn.Module):

    def __init__ (self, hidden_layers: int = 3, layer_size: int = 100):
        super(MNIST, self).__init__()
        self.pic_size = 28 * 28
        self.classes = 10

        linears = [ nn.Sequential(
                nn.Linear(layer_size, layer_size),
                nn.ReLU()
            ) for _ in range(hidden_layers)
        ]

        self.net = nn.Sequential(
            nn.Linear(self.pic_size, layer_size),
            *linears,
            nn.Linear(layer_size, self.classes),
            nn.Softmax()
        )

    def forward(self, x):
        x = x.view(-1, self.pic_size)
        x = self.net(x)
        return x

network = MNIST()

In [149]:
# datasets
BATCH_SIZE = 64

trainset = torchvision.datasets.MNIST(
    '/files/', 
    train=True,
    download=True, 
    transform=torchvision.transforms.ToTensor()
)

testset = torchvision.datasets.MNIST(
    '/files/', 
    download=True, 
    transform=torchvision.transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    testset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
)

In [153]:
def train(
    network,
    train_loader,
    criterion = nn.CrossEntropyLoss(),
    optimizer = optim.SGD,
    num_epochs = 10,
    learning_rate = .1,
):
    optimizer = optimizer(network.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch_idx, sample in enumerate(train_loader):
            pics, labels = sample
            labels = F.one_hot(labels, num_classes=10)
            optimizer.zero_grad()
            outputs = network(pics)
            loss = criterion(outputs, labels.to(torch.float))
            loss.backward()
            optimizer.step()
            epoch_loss += loss

        print(f"epoch: {epoch}, loss: {epoch_loss}")

train(network, train_loader)

epoch: 0, loss: 1451.28369140625
epoch: 1, loss: 1447.452392578125
epoch: 2, loss: 1443.673583984375
epoch: 3, loss: 1439.791259765625
epoch: 4, loss: 1435.8707275390625
epoch: 5, loss: 1437.7442626953125
epoch: 6, loss: 1433.578369140625
epoch: 7, loss: 1433.619140625
epoch: 8, loss: 1431.4893798828125
epoch: 9, loss: 1431.6839599609375


In [155]:
def test(
    network, 
    test_loader,
    criterion = nn.CrossEntropyLoss(),
): 
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): 
        for data, target in test_loader:
            output = network(data)
            correct += sum(target == output.argmax(axis=1))
            labels = F.one_hot(target, num_classes=10)
            test_loss += criterion(output, labels.to(torch.float))
        test_loss /= len(test_loader.dataset)

        print(f"Test set: avg loss {test_loss}, accuracy: {100*correct/len(test_loader.dataset):.2f}%")
    
test(network, test_loader)

Test set: avg loss 0.023818587884306908, accuracy: 93.75%


# ABS


In [154]:
print(network)

MNIST(
  (net): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=100, out_features=100, bias=True)
      (1): ReLU()
    )
    (4): Linear(in_features=100, out_features=10, bias=True)
    (5): Softmax(dim=None)
  )
)
